In [1]:
from langchain_community.document_loaders import PyMuPDFLoader

# pyMuPDFLoader 객체 정의
loader = PyMuPDFLoader("data/SPRi AI Brief_10월호_산업동향_F.pdf")

# 문서 로드
docs = loader.load()

print(f"문서의 페이지수 : {len(docs)}")

문서의 페이지수 : 25


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 61


In [3]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [4]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [5]:
for doc in vectorstore.similarity_search("메타"):
    print(doc.page_content)

1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
7
메타의 AI 모델 ‘라마’, 다운로드 수 3억 5천만 회 달성하며 활발한 생태계 형성
n 메타의 오픈소스 AI 모델 ‘라마’는 2023년 2월 출시 이래 다운로드 수 3억 5천만 건을 
기록했으며, 허깅페이스에는 6만 개 이상의 라마 파생 모델이 존재
n 액센추어, AT&T, 도어대시, 노무라, 줌을 비롯한 주요 기업들은 보고서 작성, 고객 관리, 개발 
업무 지원 등 자체 목적에 맞게 라마를 미세 조정하여 활용
KEY Contents
 
£ 메타의 오픈소스 AI 모델 ‘라마’, 2023년 대비 다운로드 수 10배 이상 증가
n 메타(Meta)가 2023년 2월 처음 출시한 AI 모델 ‘라마(Llama)’의 다운로드 수가 3억 5천만 건에 
달하며 2023년 대비 다운로드 수가 10배 이상 증가했다고 발표 
∙라마는 3.1 버전이 출시된 2024년 7월 한 달 동안 허깅페이스(Hugging Face)*에서 2천만 건 이상
의 다운로드를 기록해 오픈소스 모델 제품군 중 선두를 차지
* 머신러닝 모델을 구축, 배포, 훈련할 수 있도록 다양한 도구와 라이브러리를 제공하는 플랫폼
∙AWS, 마이크로소프트(Microsoft) 등의 대형 클라우드 제공업체를 통한 라마의 월간 사용량은 2024년 
1월~7월까지 10배 증가했으며, 2024년 8월 한 달 동안 가장 사용자 수가 많았던 버전은 ‘라마 3.1-405B’로 
최대 규모 AI 모델의 인기를 입증
2024년 10월호
1월~7월까지 10배 증가했으며, 2024년 8월 한 달 동안 가장 사용자 수가 많았던 버전은 ‘라마 3.1-405B’로 
최대 규모 AI 모델의 인기를 입증
∙자체 활용 목적에 맞게 라마를 미세 조정하는 개발자 커뮤니티가 활성화되며, 허깅페이스에는 6만 개 
이상의 라마 파생 모델이 존재
∙메타는 라마의 성공이 오픈소스에 기인한다며, 개발자의 선택권과 역량을 보장함으로써 AI 생태계가 
활성화되고 광범위하고 빠른 혁신이 가능해

In [6]:
# 벡터스토어에 있는 정보를 검색하고 생성
retriever = vectorstore.as_retriever()

In [8]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question.
    Use a very kind and gentle tone like a kindergarten teacher talking to a child.
    Speak in a warm and friendly way.
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.

    #Context: 
    {context}

    #Question:
    {question}

    #Answer:
    """
)

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context" : retriever, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
# 체인 실행
question = "un에서 발표한 최종보고서 내용 요약해주"
response = chain.invoke(question)

print(response)

안녕, 친구야! UN에서 발표한 최종 보고서에 대해 이야기해줄게. 이 보고서는 '인류를 위한 AI 거버넌스'라는 주제로 만들어졌어. 보고서에서는 AI의 위험을 해결하고, 그 혁신적인 잠재력을 잘 활용하기 위해 국제 협력을 바탕으로 한 포용적이고 분산된 AI 거버넌스 체계를 마련하자고 촉구했단다. 그리고 AI 국제과학 패널 구성, AI 정책 대화, AI 표준 교류와 AI 역량 개발 네트워크 형성 등 7가지 권고안을 제시했어. 이 모든 것들은 AI가 사람들에게 더 좋은 영향을 미칠 수 있도록 돕기 위한 것이란다. 궁금한 게 더 있으면 언제든지 물어봐!
